In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import pandas as pd
import numpy as np
import plotly.express as px 
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import haystack_utils
import hook_utils
import plotting_utils
import probing_utils
from probing_utils import get_and_score_new_word_probe
from sklearn import preprocessing

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

model = HookedTransformer.from_pretrained("EleutherAI/pythia-160m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

german_data = haystack_utils.load_json_data("data/german_europarl.json")[:200]
english_data = haystack_utils.load_json_data("data/english_europarl.json")[:200]

LAYER, NEURON = 8, 2994

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


In [14]:
with open(f'data/pythia_160m/layer_8/single_neurons_df.pkl', 'rb') as f:
    one_sparse_probe_scores_df = pickle.load(f)

top_one_scores = one_sparse_probe_scores_df.sort_values(by='mcc', ascending=False).head(10)
print(top_one_scores)

with open(f'data/pythia_160m/layer_8/probes_two_sparse_df_10_mcc.pkl', 'rb') as f:
    two_sparse_probe_scores_df = pickle.load(f)

top_two_scores = two_sparse_probe_scores_df.sort_values(by='mcc', ascending=False).head(10)
print(top_two_scores)


            f1       mcc
1426  0.859275  0.742336
149   0.795104  0.641232
2994  0.817568  0.637815
1815  0.803144  0.625381
1080  0.821160  0.624540
626   0.760399  0.622951
2288  0.789410  0.621133
2636  0.811525  0.614484
1404  0.816850  0.613347
2830  0.775984  0.608591
            f1       mcc  neuron_1  neuron_2
1506  0.896530  0.800370      1426      1507
2074  0.889679  0.787044      1426      2075
11    0.889597  0.785888      1426        11
2443  0.889129  0.785524      1426      2444
2029  0.887817  0.785415      1426      2030
1154  0.887900  0.784629      1426      1154
2741  0.886871  0.783621      1426      2742
1921  0.887128  0.781480      1426      1922
45    0.886047  0.781297      1426        45
667   0.881716  0.779885      1426       667
